<a href="https://colab.research.google.com/github/achri19/BAM/blob/main/Downloading_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size="8">Preparations: Downloading Ancillary Data</font>

<font size="3">In this notebook, we will:

- Download GEBCO, Global Mangrove Watch, and Geoid datasets that are needed for the tutorials. They will take up 8-15GB of space, so make sure you have room.



<font size=5 color='green'> If you are running in Google Colab, set the variable yes_colab = True.</font> <br>
<font size=5 color='blue'> If you are running on your own computer, set the variable yes_colab = False </font>


In [1]:
yes_colab = True


<font size=6> Step #1: Set working directory <br> </font>



<font size=5 color='green'> If you are running in Google Colab, when you run the next cell, a pop-up window will appear asking you to grant access to your Google Drive. You must approve or the notebook will not work. <font> <br>



In [2]:
import os
import sys
from pathlib import Path

your_path = Path(os.getcwd() + '/')
if yes_colab:
  where_to_mount = '/content/drive/'
  from google.colab import drive
  drive.mount(where_to_mount, force_remount=True)
  mounted_drive = Path(where_to_mount) / 'MyDrive' 
  sys.path.append(str(mounted_drive / 'installations'))
  path_ancillary = mounted_drive / 'ancillary'
  Path(mounted_drive / 'installations').mkdir(parents=True, exist_ok=True)

else:
  mounted_drive = Path(os.path.abspath(os.path.join(your_path, os.pardir)))
  path_ancillary = mounted_drive / 'BAM/ancillary'

print('Your working directory is %s' %(mounted_drive))



Mounted at /content/drive/
Your working directory is /content/drive/MyDrive


<font size=6> Step #2: Install and import packages. <font> <br>

<font size=5 color='green'> If you are running in Google Colab, this cell should install all Python packages you need for each tutorial. </font> <br>
<font size=5 color='blue'> If you are running on your own computer, the packages were already installed when you installed the conda environment </font>


In [3]:
import os
os.chdir(mounted_drive)
if yes_colab:
  if os.path.isdir(mounted_drive / 'BAM'):
    print('## Updating the local git repository \n')
    os.chdir(mounted_drive / 'BAM')
    #!git fsck --full
    # !git add -A 
    !git stash   
    !git pull
    # !rm -rf {mounted_drive/'BAM'}

  else:
    print('## Pulling the git repository with files for the tutorial\n')
    !git clone https://github.com/achri19/BAM.git


## Updating the local git repository 

Saved working directory and index state WIP on main: 42f28f2 updated
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), 78.04 KiB | 54.00 KiB/s, done.
From https://github.com/achri19/BAM
   42f28f2..7ac1877  main       -> origin/main
Updating 42f28f2..7ac1877
Fast-forward
 Download_Ancillary.ipynb          |  39 +++++
 notebooks/Build_Model_Files.ipynb | 340 +++++++++++++++++++++++---------------
 2 files changed, 242 insertions(+), 137 deletions(-)
 create mode 100644 Download_Ancillary.ipynb


In [4]:
if yes_colab:
  print('\n## Installing the Python packages needed for these tutorials\n')
  try:
    import geopandas as gpd
  except:
    !/bin/bash $mounted_drive/BAM/notebooks/install_packages_colab_debug.sh



## Installing the Python packages needed for these tutorials



In [5]:
path_ancillary = mounted_drive / 'BAM/ancillary'
path_code = mounted_drive / 'BAM/scripts/'
path_templates = mounted_drive /  'BAM/templates/'
path_configs = mounted_drive / 'BAM/configs/'
sys.path.insert(1,str(path_code))
print(path_code)
print(path_ancillary)

path_examples = mounted_drive / 'BAM/examples'
Path(path_examples).mkdir(parents=True, exist_ok=True)

/content/drive/MyDrive/BAM/scripts
/content/drive/MyDrive/BAM/ancillary


In [ ]:
import sys
import os
import pandas as pd
import shutil
from datetime import datetime
from string import Template
import fnmatch
import geopandas as gpd
import rasterio
from osgeo import gdal 
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import rtree
from scipy import *


In [ ]:
try:
  import pyTMD
except:
  !pip install pytmd

from main_tools import (build_directory,
                       get_extent_parameters,
                       setup_AOI_files, 
                       make_polygons,
                       make_channel_networks,
                       make_model_foundation, 
                       set_boundary_conditions, 
                       make_watermask,
                       more_opening)


<font size=6> Step #3: Download to your ancillary folder <br> </font>



In [ ]:
import urllib.request
if os.path.isfile(path_ancillary / 'geoids' / 'egm08_25.gtx')==False:

  url = 'https://download.osgeo.org/proj/vdatum/egm08_25/egm08_25.gtx'
  filename = path_ancillary / 'geoids' / 'egm08_25.gtx'
  urllib.request.urlretrieve(url, filename)


if os.path.isfile(path_ancillary / 'gebco' / 'gebco_all.vrt')==False:
  url = 'https://www.bodc.ac.uk/data/open_download/gebco/gebco_2022_sub_ice_topo/geotiff/'
  filename = path_ancillary / 'gebco' / 'gebco.zip'
  urllib.request.urlretrieve(url, filename)

  with zipfile.ZipFile(path_ancillary / 'gebco' / 'gebco.zip', 'r') as zip_ref:
      zip_ref.extractall(path_ancillary / 'gebco' )

  gebco_files = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(path_ancillary / 'gebco')
                for f in fnmatch.filter(files,'*gebco_2022*.tif')]
  print(gebco_files)

  with open(path_ancillary / 'gebco' / 'gebco_files.txt', 'w') as f:
    for item in gebco_files:
      f.write("%s\n" % item)
  ! gdalbuildvrt -input_file_list {path_ancillary}/gebco/gebco_files.txt {path_ancillary}/gebco/gebco_all.vrt
  !rm {path_ancillary}/'gebco'/'gebco.zip'

if os.path.isfile(path_ancillary / 'gmw' / 'gmw_v3_2016_vec.shp')==False:
  url = 'https://zenodo.org/record/6894273/files/gmw_v3_2016_vec.zip?download=1'
  filename = path_ancillary / 'gmw' / 'gmw_v3_2016_vec.zip'
  urllib.request.urlretrieve(url, filename)

  with zipfile.ZipFile(path_ancillary / 'gmw' / 'gmw_v3_2016_vec.zip', 'r') as zip_ref:
      zip_ref.extractall(path_ancillary / 'gmw')

!rm {path_ancillary}/'gmw'/'gmw_v3_2016_vec.zip'


In [ ]:

ref_10m = rasterio.open('%s_GEBCO_%s.tif' %(folders[8] / AOI,res))

ref_10m,parameters = get_extent_parameters(path_ancillary,AOI,folders,res,parameters)





##############################################################################################
##############################[Step 1b][GetExtentParameters]##################################
##############################################################################################

##################### AOI extent is set by: komo_input.shp
The input shapefile is not in the correction projection (EPSG 4326), reprojecting to EPSG 4326

[Step 2][Setup_AOI_Files][Determine EPSG code and UTM zone] .......

##################### ANUGA Models must be in UTM
##################### UTM Zone: 32n
##################### EPSG: 32232 

[Step 2][Setup_AOI_Files][Extending AOI by 1000m] .......


[Step 2][Setup_AOI_Files][Setting up AOI extent] .......

##################### AOI bounds are : 501790, -2260, 610150, 64700
##################### Approximate area of AOI extent is 7256.0 km^2

[Step 2][Setup_AOI_Files][Downloading GEBCO Dataset as reference for projection, resolution, etc] .......


[S